In [ ]:
import sire as sr
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dihedral = "1234"
seed_no = 4
fragment = 5

In [ ]:
n_confs = 500 # number of conformers scanned

In [ ]:
idx1 = dihedral[0] # will need to change to automatically parse torsion idx
idx2 = dihedral[1]
idx3 = dihedral[2]
idx4 = dihedral[3] 

In [ ]:
forward = np.arange(-180,190,10).tolist()

In [ ]:
energies_per_dihedral = []

In [ ]:
conf_numbers = np.arange(0,n_confs-1).tolist() # update to use conformers number when the dihedral is specified

In [ ]:
energies_per_dihedral = []

for dihe in forward:
    print('Calculating MM energies for ', dihe, 'degrees')
    mm_energies = []
    file_patterns_prmtop = ['../profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf%s_dihe_%s*_resp_charges_atomtypes_renamed.mol2.prmtop' % (fragment, dihedral, dihedral, seed_no, conf, dihe) for conf in conf_numbers]
    file_patterns_inpcrd = ['../profiles_torsions/optimised_geometries/fragment%s/torsion%s/torsion%s_seed%s_conf%s_dihe_%s*_resp_charges_atomtypes_renamed.mol2.inpcrd' % (fragment, dihedral, dihedral, seed_no, conf, dihe) for conf in conf_numbers]

    file_list_prmtop = []
    file_list_inpcrd = []

    # Iterate through file patterns and find matching files
    for pattern_prmtop, pattern_inpcrd in zip(file_patterns_prmtop, file_patterns_inpcrd):
        file_list_prmtop.extend(glob.glob(pattern_prmtop))
        file_list_inpcrd.extend(glob.glob(pattern_inpcrd))
    
    for fle in file_list_inpcrd:
        mol = sr.load(file_list_prmtop[0], fle)
        torsion = mol.dihedrals(f"atomnum {idx1}", f"atomnum {idx2}", f"atomnum {idx3}", f"atomnum {idx4}")
        mm_energies.append(mol.energy().value()-torsion.energy().value()) # energies in kcal/mol 
    #print(mm_energies)
    energies_per_dihedral.append(mm_energies)

#print(energies_per_dihedral)

In [ ]:
torsion = mol.dihedrals(f"atomnum {idx1}", f"atomnum {idx2}", f"atomnum {idx3}", f"atomnum {idx4}")
torsion.potentials() # this shows us the current GAFF2 expression for the torsion

In [ ]:
energies_per_dihedral

In [ ]:
# check if values for 180 and -180 are missing
assert energies_per_dihedral[0] == []


In [ ]:
assert energies_per_dihedral[-1] == []

In [ ]:
avg_energy_per_dihedral = [] # saving average energy per dihedral for all conformers

for i in energies_per_dihedral: 
    avg_energy_per_dihedral.append(np.mean(i))

avg_energy_per_dihedral

In [ ]:
# normalise average energy

norm_avg_energy_per_dihedral = []

for enrg in avg_energy_per_dihedral[1:]: # usually my -180 never gets scanned so omit
    norm_avg_energy_per_dihedral.append(enrg-np.min(avg_energy_per_dihedral[1:]))

len(norm_avg_energy_per_dihedral)
norm_avg_energy_per_dihedral

In [ ]:
# we need to subtract this array from the QM profile

np.save('../profiles_torsions/individual_conformer_scans/fragment%s/torsion%s/torsion%s_seed%s_mm.npy' % (fragment, dihedral, dihedral, seed_no), norm_avg_energy_per_dihedral)

In [ ]:
idx = 0

for i in norm_avg_energy_per_dihedral:
    plt.plot(forward[idx], i, marker='o', linestyle='None', color='blue')
    idx+=1
plt.xlabel('Dihedral Angle (degrees)')
plt.ylabel('Total Energy - Torsion Energy (kcal/mol)')
plt.savefig('../profiles_torsions/individual_conformer_scans/fragment%s/torsion%s/mm_total_energy_minus_torsion%s_seed%s.png' % (fragment, dihedral, dihedral, seed_no))